## Rank Averaging

In [1]:
import pandas as pd
import numpy as np

In [2]:
data=pd.read_csv('train_cleaned.csv')

In [3]:
data.shape

(8523, 46)

In [4]:
data.head()

,Item_Weight,Item_Visibility,Item_MRP,Outlet_Establishment_Year,Item_Outlet_Sales,Item_Fat_Content_LF,Item_Fat_Content_Low Fat,Item_Fat_Content_Regular,Item_Fat_Content_low fat,Item_Fat_Content_reg,...,Outlet_Size_High,Outlet_Size_Medium,Outlet_Size_Small,Outlet_Location_Type_Tier 1,Outlet_Location_Type_Tier 2,Outlet_Location_Type_Tier 3,Outlet_Type_Grocery Store,Outlet_Type_Supermarket Type1,Outlet_Type_Supermarket Type2,Outlet_Type_Supermarket Type3
0,9.30,0.016047,249.8092,1999,3735.1380,0,1,0,0,0,...,0,1,0,1,0,0,0,1,0,0
1,5.92,0.019278,48.2692,2009,443.4228,0,0,1,0,0,...,0,1,0,0,0,1,0,0,1,0
2,17.50,0.016760,141.6180,1999,2097.2700,0,1,0,0,0,...,0,1,0,1,0,0,0,1,0,0
3,19.20,0.000000,182.0950,1998,732.3800,0,0,1,0,0,...,0,0,0,0,0,1,1,0,0,0
4,8.93,0.000000,53.8614,1987,994.7052,0,1,0,0,0,...,1,0,0,0,0,1,0,1,0,0


In [5]:
#seperating independent and dependent variables
x = data.drop(['Item_Outlet_Sales'], axis=1)
y = data['Item_Outlet_Sales']

In [6]:
from sklearn.model_selection import train_test_split
train_x,valid_x,train_y,valid_y = train_test_split(x,y, random_state = 101, shuffle=False)

In [7]:
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor

In [8]:
model1 = LinearRegression()
model1.fit(train_x,train_y)
pred1=model1.predict(valid_x)
pred1[:10], model1.score(valid_x, valid_y)

(array([ 2443.32513354,  2788.36657095,  2169.73477778,  2385.33394477,
         2716.58813899,  3771.41850023,  5119.35227101,  4133.40997069,
        -1010.95412123,  1365.9929378 ]),
 0.5551582433748833)

In [9]:
model2 = KNeighborsRegressor(n_neighbors=9)
model2.fit(train_x,train_y)
pred2=model2.predict(valid_x)
pred2[:10], model2.score(valid_x, valid_y)

(array([1859.06155556, 2519.09128889, 2620.88471111, 2277.33191111,
        3801.12617778, 4099.10866667, 5151.88642222, 5378.62831111,
         476.7128    , 1501.15706667]),
 0.5007936087911619)

In [10]:
model3 = DecisionTreeRegressor(max_depth=7)
model3.fit(train_x,train_y)
pred3=model3.predict(valid_x)
pred3[:10], model3.score(valid_x, valid_y)

(array([2413.81776312, 2822.71913115,  585.22631071, 2348.86112052,
        3153.22486811, 3544.38935413, 5815.65707727, 5048.681504  ,
         105.25188333, 1595.93215694]),
 0.5735386013514847)

In [11]:
m1_score= model1.score(valid_x, valid_y)
m2_score= model2.score(valid_x, valid_y)
m3_score= model3.score(valid_x, valid_y)
m1_score, m2_score, m3_score

(0.5551582433748833, 0.5007936087911619, 0.5735386013514847)

In [12]:
index_ = [1,2,3]
valid_r2 = [m1_score,m2_score,m3_score]

rank_eval = pd.DataFrame({
    'score':valid_r2
}, index = index_)
rank_eval

,score
1,0.555158
2,0.500794
3,0.573539


In [13]:
sorted_rank = rank_eval.sort_values('score')
sorted_rank

,score
2,0.500794
1,0.555158
3,0.573539


In [14]:
sorted_rank['rank'] = [i for i in range(1,4)]
sorted_rank

,score,rank
2,0.500794,1
1,0.555158,2
3,0.573539,3


In [15]:
sorted_rank['weight'] = sorted_rank['rank']/sorted_rank['rank'].sum()
sorted_rank

,score,rank,weight
2,0.500794,1,0.166667
1,0.555158,2,0.333333
3,0.573539,3,0.500000


In [16]:
wt_pred1 = pred1*float(sorted_rank.loc[[1],['weight']].values)
wt_pred2 = pred2*float(sorted_rank.loc[[2],['weight']].values)
wt_pred3 = pred3*float(sorted_rank.loc[[3],['weight']].values)
ranked_prediction = wt_pred1 + wt_pred2 + wt_pred3
ranked_prediction

array([2331.19418533, 2760.66363737, 1452.6721998 , ..., 1348.78554185,
       1570.52823782, 1268.26795746])

In [18]:
from sklearn.metrics import r2_score
r2_score(valid_y, ranked_prediction)

0.5828295343429305